In [1]:
from baseline_cnn import *
from baseline_cnn import BasicCNN

In [2]:
# Setup: initialize the hyperparameters/variables
num_epochs = 1           # Number of full passes through the dataset
batch_size = 16          # Number of samples in each minibatch
learning_rate = 0.001  
seed = np.random.seed(1) # Seed the random number generator for reproducibility
p_val = 0.1              # Percent of the overall dataset to reserve for validation
p_test = 0.2             # Percent of the overall dataset to reserve for testing

#TODO: Convert to Tensor - you can later add other transformations, such as Scaling here
transform = transforms.Compose([transforms.Resize([512,512], interpolation=2),transforms.ToTensor()])


# Check if your system supports CUDA
use_cuda = torch.cuda.is_available()

# Setup GPU optimization if CUDA is supported
if use_cuda:
    computing_device = torch.device("cuda")
    extras = {"num_workers": 1, "pin_memory": True}
    print("CUDA is supported")
else: # Otherwise, train on the CPU
    computing_device = torch.device("cpu")
    extras = False
    print("CUDA NOT supported")

# Setup the training, validation, and testing dataloaders
train_loader, val_loader, test_loader = create_split_loaders(batch_size, seed, transform=transform, 
                                                             p_val=p_val, p_test=p_test,
                                                             shuffle=True, show_sample=False, 
                                                             extras=extras)

# Instantiate a BasicCNN to run on the GPU or CPU based on CUDA support
model = BasicCNN()
model = model.to(computing_device)
print("Model on CUDA?", next(model.parameters()).is_cuda)

#TODO: Define the loss criterion and instantiate the gradient descent optimizer
criterion = torch.nn.MultiLabelMarginLoss(size_average=True)
#TODO - loss criteria are defined in the torch.nn package

#TODO: Instantiate the gradient descent optimizer - use Adam optimizer with default parameters
optimizer = optim.Adam(model.parameters(),lr=learning_rate) #TODO - optimizers are defined in the torch.optim package

CUDA is supported
Model on CUDA? True


/opt/conda/lib/python3.6/site-packages/torch/nn/_reduction.py:49: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


In [3]:
# Track the loss across training
total_loss = []
avg_minibatch_loss = []

# Begin training procedure
for epoch in range(num_epochs):

    N = 50
    N_minibatch_loss = 0.0    

    # Get the next minibatch of images, labels for training
    for minibatch_count, (images, labels) in enumerate(train_loader, 0):

        # Put the minibatch data in CUDA Tensors and run on the GPU if supported
        images, labels = images.to(computing_device), labels.to(computing_device)
        labels = labels.long() 
        # Zero out the stored gradient (buffer) from the previous iteration
        optimizer.zero_grad()

        # Perform the forward pass through the network and compute the loss
        outputs = model(images)

        loss = criterion(outputs,labels)
        
        # Automagically compute the gradients and backpropagate the loss through the network
        loss.backward()

        # Update the weights
        optimizer.step()

        # Add this iteration's loss to the total_loss
        total_loss.append(loss.item())
        N_minibatch_loss += loss
        
        #TODO: Implement validation
        
        if minibatch_count % N == 0:    
            
            # Print the loss averaged over the last N mini-batches    
            N_minibatch_loss /= N
            print('Epoch %d, average minibatch %d loss: %.3f' %
                (epoch + 1, minibatch_count, N_minibatch_loss))
            
            # Add the averaged loss over N minibatches and reset the counter
            avg_minibatch_loss.append(N_minibatch_loss)
            N_minibatch_loss = 0.0
        
    print("Finished", epoch + 1, "epochs of training")
print("Training complete after", epoch, "epochs")

Epoch 1, average minibatch 0 loss: 0.253
Epoch 1, average minibatch 50 loss: 12.069
Epoch 1, average minibatch 100 loss: 11.679
Epoch 1, average minibatch 150 loss: 11.300
Epoch 1, average minibatch 200 loss: 11.004
Epoch 1, average minibatch 250 loss: 10.698
Epoch 1, average minibatch 300 loss: 10.312
Epoch 1, average minibatch 350 loss: 10.026
Epoch 1, average minibatch 400 loss: 9.821
Epoch 1, average minibatch 450 loss: 9.541
Epoch 1, average minibatch 500 loss: 9.206
Epoch 1, average minibatch 550 loss: 8.891
Epoch 1, average minibatch 600 loss: 8.648
Epoch 1, average minibatch 650 loss: 8.319
Epoch 1, average minibatch 700 loss: 8.137
Epoch 1, average minibatch 750 loss: 7.904
Epoch 1, average minibatch 800 loss: 7.712
Epoch 1, average minibatch 850 loss: 7.407
Epoch 1, average minibatch 900 loss: 7.220
Epoch 1, average minibatch 950 loss: 7.124
Epoch 1, average minibatch 1000 loss: 6.808
Epoch 1, average minibatch 1050 loss: 6.643
Epoch 1, average minibatch 1100 loss: 6.466
Epoc

In [4]:
import torch
print(torch.__version__)

1.0.0
